In [19]:
import numpy as np
real_index = []
for i in range (1,41):
    j= str(i)
    index = 'data'+ j +'.avi'
    # print(index)
    real_index.append(index)

# print(real_index)
real_index.remove('data28.avi')
print(real_index)

['data1.avi', 'data2.avi', 'data3.avi', 'data4.avi', 'data5.avi', 'data6.avi', 'data7.avi', 'data8.avi', 'data9.avi', 'data10.avi', 'data11.avi', 'data12.avi', 'data13.avi', 'data14.avi', 'data15.avi', 'data16.avi', 'data17.avi', 'data18.avi', 'data19.avi', 'data20.avi', 'data21.avi', 'data22.avi', 'data23.avi', 'data24.avi', 'data25.avi', 'data26.avi', 'data27.avi', 'data29.avi', 'data30.avi', 'data31.avi', 'data32.avi', 'data33.avi', 'data34.avi', 'data35.avi', 'data36.avi', 'data37.avi', 'data38.avi', 'data39.avi', 'data40.avi']


In [20]:
# imports
import cv2
import mediapipe as mp
import torch
import torchvision.models as models
import torchvision
import torchvision.transforms as transforms
import numpy

import torch
from torchvision.models import resnet18
import numpy as np


In [21]:
All_features = []
for k in real_index:
    print(k)
    # Step 1: Load the video file and extract each frame
    cap = cv2.VideoCapture(k)

    # Step 2: Initialize MediaPipe's face detection model
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection()

    # Step 3: Detect faces in each frame and select ROI for each face
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the image to RGB format for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect faces using MediaPipe's face detection model
        results = face_detection.process(frame_rgb)

        # Iterate through each detected face and select ROI
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            x, y, w, h = int(bbox.xmin * w), int(bbox.ymin * h), int(bbox.width * w), int(bbox.height * h)

            # Select ROI for each face (forehead, cheek, and nose)
            forehead_roi = frame[y:y+h//3, x:x+w]
            cheek_roi = frame[y+h//3:y+2*h//3, x:x+w]
            nose_roi = frame[y+2*h//3:y+h, x:x+w]

            # Draw bounding box and ROI on the frame
            # cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h//3), (255, 0, 0), 2) # Forehead ROI
            cv2.rectangle(frame, (x, y+h//3), (x+w, y+2*h//3), (0, 0, 255), 2) # Cheek ROI
            cv2.rectangle(frame, (x, y+2*h//3), (x+w, y+h), (255, 255, 0), 2) # Nose ROI

        # Show the frame with detected face area and ROI area
        # cv2.imshow('frame', frame)
        # if cv2.waitKey(1) == ord('q'):
        #     break

    # Release the video capture and destroy all windows
    cap.release()
    cv2.destroyAllWindows()
#################################################
    # using Resnet 18
    resnet18 = torchvision.models.resnet18()
    modules = list(resnet18.children())[: -4]   # Select up to the 14th layer
    model = torch.nn.Sequential(*modules)

    # Step 2: Normalize the pixel values in each map
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    # Step 3: Extract spatial-temporal features from ROIs in each frame
    feature_maps = []
    cap = cv2.VideoCapture(k)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the ROIs to PyTorch tensor and normalize the pixel values
        forehead_tensor = normalize(transforms.ToTensor()(forehead_roi)).unsqueeze(0)
        cheek_tensor = normalize(transforms.ToTensor()(cheek_roi)).unsqueeze(0)
        # nose_tensor = normalize(transforms.ToTensor()(nose_roi)).unsqueeze(0)

        # Extract features using the pre-trained ResNet18 model
        with torch.no_grad():
            forehead_features = model(forehead_tensor).squeeze().numpy()
            cheek_features = model(cheek_tensor).squeeze().numpy()
            # nose_features = model(nose_tensor).squeeze().numpy()

        # Step 2: Normalize the pixel values in each map to have a mean of zero and a standard deviation of one
        forehead_features = (forehead_features - np.mean(forehead_features)) / np.std(forehead_features)
        cheek_features = (cheek_features - np.mean(cheek_features)) / np.std(cheek_features)
        # nose_features = (nose_features - np.mean(nose_features)) / np.std(nose_features)

        # Step 3: Create a spatial-temporal map by combining the normalized maps
        spatial_temporal_map = np.concatenate((forehead_features, cheek_features), axis=0)
        feature_maps.append(spatial_temporal_map)

    # Convert the feature maps to a NumPy array
    feature_maps = np.array(feature_maps)

    # Release the video capture and destroy all windows
    cap.release()
    cv2.destroyAllWindows()
#################################################
    print(feature_maps.shape)
    All_features.append(feature_maps)

data1.avi
(1205, 256, 6, 18)
data2.avi
(1208, 256, 7, 20)
data3.avi
(1207, 256, 8, 23)
data4.avi
(1209, 256, 7, 20)
data5.avi
(1210, 256, 7, 21)
data6.avi
(1208, 256, 8, 22)
data7.avi
(1209, 256, 7, 19)
data8.avi
(1210, 256, 8, 23)
data9.avi
(1225, 256, 7, 20)
data10.avi
(1227, 256, 8, 23)
data11.avi
(1226, 256, 6, 18)
data12.avi
(1577, 256, 6, 18)
data13.avi
(1221, 256, 8, 23)
data14.avi
(1248, 256, 6, 18)
data15.avi
(1298, 256, 7, 19)
data16.avi
(1232, 256, 8, 22)
data17.avi
(1226, 256, 7, 21)
data18.avi
(1243, 256, 8, 24)
data19.avi
(1209, 256, 7, 19)
data20.avi
(1209, 256, 9, 25)
data21.avi
(1208, 256, 7, 20)
data22.avi
(1209, 256, 7, 21)
data23.avi
(1208, 256, 7, 21)
data24.avi
(1209, 256, 9, 26)
data25.avi
(1233, 256, 7, 21)
data26.avi
(1230, 256, 7, 21)
data27.avi
(1227, 256, 6, 18)
data29.avi
(2154, 256, 8, 22)
data30.avi
(1231, 256, 7, 22)
data31.avi
(1245, 256, 6, 18)
data32.avi
(1217, 256, 7, 19)
data33.avi
(1216, 256, 7, 19)
data34.avi
(1224, 256, 7, 20)
data35.avi
(1235, 2

In [30]:
# All_features = torch.tensor(All_features)
All_feature_size = []
for i in All_features:
    a = i[0:1200 , : , 0:6, 0:18 ]
    All_feature_size.append(a)
    # print(a.shape) #1200*128


print(len(All_feature_size))
print(All_feature_size[0].shape)
All_feature_size = torch.tensor(All_feature_size)

39
(1200, 256, 6, 18)


In [31]:
import torch
import torch.nn as nn
All_flattened = []
num_channels = 256 #(num_frames, num_channels, height, width)

for i in All_feature_size:

    class ConvNet(nn.Module):
        def __init__(self):
            super(ConvNet, self).__init__()

            self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=16, kernel_size=3, stride=1, padding=1)
            self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
            self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

            self.flatten = nn.Flatten()

        def forward(self, x):
            x = self.conv1(x)
            x = nn.functional.relu(x)
            x = self.pool1(x)
            x = self.conv2(x)
            x = nn.functional.relu(x)
            x = self.pool2(x)
            x = self.flatten(x)

            flattened_features = x

            return flattened_features

    conv_net = ConvNet()
    stacked_tensor = i
    flattened_features = conv_net(stacked_tensor)
    print(flattened_features.shape)
    All_flattened.append(flattened_features)

torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])
torch.Size([1200, 128])


In [38]:
#39 members
HR = [84,71,70,76,73,63,61,72,75,84,67,73,68,86,91,60,64,81,69,74,83,61,83,95,90,88,66,96,58,56,61,76,62,88,92,63,64,60,64]
print(len(HR))

HR_tensor = torch.tensor(HR)
print(HR_tensor.shape)
HR_tensor_float = HR_tensor.float()

39
torch.Size([39])


In [39]:
data_3d = torch.stack(All_flattened) #(num_samples or batches, num_frames, flattened_feature_size)
print(data_3d.shape)

data_3d_detach = data_3d.detach()
print(data_3d_detach.shape)  #(num_samples, num_frames, flattened_feature_size)

torch.Size([39, 1200, 128])
torch.Size([39, 1200, 128])


In [40]:
# LSTM MODEL
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split

# set random seed for reproducibility
# torch.manual_seed(42)

# define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# define the hyperparameters
input_size = 128   #flattened_feature_size
hidden_size = 256
num_layers = 1
output_size = 1
lr = 0.001
num_epochs = 30
batch_size = 1

# split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(data_3d_detach, HR_tensor_float, test_size=0.3, random_state=42)
# Split the data into train and test sets
num_samples = 40
train_size = int(0.7 * num_samples)
test_size = num_samples - train_size
data_train = data_3d_detach[:train_size]
data_test = data_3d_detach[train_size:]
HR_train = HR_tensor_float[:train_size]
HR_test = HR_tensor_float[train_size:]

# convert the data to PyTorch tensors
X_train = data_train
y_train = HR_train
X_test = data_test
y_test = HR_test

# create data loaders for batch processing
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# create the model and optimizer
model = LSTM(input_size, hidden_size, num_layers, output_size)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# criterion = nn.L1Loss()
criterion = nn.MSELoss()

# train the model
for epoch in range(num_epochs):
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print("Epoch [{}/{}], Train Loss: {:.4f}".format(epoch+1, num_epochs, train_loss/len(train_loader)))

# evaluate the model
test_loss = 0.0
predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        predictions.extend(outputs.detach().numpy())
mae = np.mean(np.abs(np.array(predictions) - y_test.numpy()))
mape = np.mean(np.abs(np.array(predictions) - y_test.numpy()) / np.array(y_test.numpy()))*100

print("Test Loss: {:.4f}, MAE: {:.4f}, MAPE: {:.4f}".format(test_loss/len(test_loader), mae, mape))


C:\Users\mojta\anaconda3\envs\cudatest\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/30], Train Loss: 4864.2042
Epoch [2/30], Train Loss: 3556.1870
Epoch [3/30], Train Loss: 2743.9351
Epoch [4/30], Train Loss: 2110.7033
Epoch [5/30], Train Loss: 1613.3036
Epoch [6/30], Train Loss: 1222.3968
Epoch [7/30], Train Loss: 914.7870
Epoch [8/30], Train Loss: 681.9811
Epoch [9/30], Train Loss: 509.8370
Epoch [10/30], Train Loss: 384.4843
Epoch [11/30], Train Loss: 294.8629
Epoch [12/30], Train Loss: 232.0637
Epoch [13/30], Train Loss: 188.9836
Epoch [14/30], Train Loss: 160.0762
Epoch [15/30], Train Loss: 141.1175
Epoch [16/30], Train Loss: 128.9764
Epoch [17/30], Train Loss: 121.3942
Epoch [18/30], Train Loss: 116.7857
Epoch [19/30], Train Loss: 114.0685
Epoch [20/30], Train Loss: 112.5229
Epoch [21/30], Train Loss: 111.6831
Epoch [22/30], Train Loss: 111.2557
Epoch [23/30], Train Loss: 111.0608
Epoch [24/30], Train Loss: 110.9915
Epoch [25/30], Train Loss: 110.9859
Epoch [26/30], Train Loss: 111.0097
Epoch [27/30], Train Loss: 111.0447
Epoch [28/30], Train Loss: 111.